In [ ]:
import tensorflow as tf
print('Tensorflow Version:', tf.__version__)

Tensorflow Version: 2.9.2


In [ ]:
msg = tf.constant('Hello Tensorflow!')
tf.print(msg)

Hello Tensorflow!


In [ ]:
import keras
print('Keras Version:', keras.__version__)

Keras Version: 2.9.0


In [ ]:
from keras import datasets
from keras import models
from keras import layers
from keras.utils import to_categorical

# Load MNIST data
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Creating network consists of two dense layers(fully connected layer)
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,), name="layer1"))
network.add(layers.Dense(10, activation='softmax', name="layer2"))

# Preparing the training images and labels
train_images = train_images.reshape(60000, 28*28)
train_images = train_images.astype('float32') / 255
train_labels = to_categorical(train_labels)

# Preparing the testing images and labels
test_images = test_images.reshape(10000, 28*28)
test_images = test_images.astype('float32') / 255
test_labels = to_categorical(test_labels)

# Prepare the network --> rmsprop : root mean square gradient descent
network.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the neural network
network.fit(train_images, train_labels, epochs=5, batch_size=128)

# Evaluate the network performance
test_loss, test_acc = network.evaluate(test_images, test_labels)

# Print testing accuracy and testing loss
print('Testing : ', test_acc)
print('Training : ', test_loss)

Epoch 1/5
469/469 [==============================] - 6s 11ms/step - loss: 0.2545 - accuracy: 0.9252
Epoch 2/5
469/469 [==============================] - 6s 13ms/step - loss: 0.1036 - accuracy: 0.9690
Epoch 3/5
469/469 [==============================] - 7s 14ms/step - loss: 0.0689 - accuracy: 0.9790
Epoch 4/5
469/469 [==============================] - 6s 13ms/step - loss: 0.0501 - accuracy: 0.9856
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.0675 - accuracy: 0.9788
Testing :  0.9787999987602234
Training :  0.06750303506851196


In [ ]:
import torch as t
import torchvision.datasets as datasets 
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt

In [ ]:
import torch
import math


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# For this example, the output y is a linear function of (x, x^2, x^3), so
# we can consider it as a linear layer neural network. Let's prepare the
# tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# In the above code, x.unsqueeze(-1) has shape (2000, 1), and p has shape
# (3,), for this case, broadcasting semantics will apply to obtain a tensor
# of shape (2000, 3) 

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. The Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
# The Flatten layer flatens the output of the linear layer to a 1D tensor,
# to match the shape of `y`.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
for t in range(2000):

    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(xx)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

# You can access the first layer of `model` like accessing the first item of a list
linear_layer = model[0]

# For linear layer, its parameters are stored as `weight` and `bias`.
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')